## Relatório do Projeto Ataxx ##

### 18/03/21 ###

***

### Ata ###

- Primeiro meeting de grupo para tomada de decisões quanto ao approach do projeto
- Criação de um __[Trello](https://trello.com/b/xQCPqqou/attaxx-projeto)__ para organização dos passos a tomar no projeto
- Início da __[pesquisa](https://docs.google.com/document/d/1fHmJtZjkOgVH6Xd6uIpjCX39uwu6PEftnRLfNoiGT4U)__ quanto aos seguintes tópicos:
  - O que é Ataxx
  - Regras do jogo
  - Objetivos do trabalho
- Início da programação do jogo base (python com a libraria pygame)

***

### Programação ###

Até agora, foi-se programado o tabuleiro base, a colocação das peças (e blocos cinzentos)segundo uma matriz dentro de uma classe que guarda o estado do jogo, os eventos de fechar o jogo e de receber cliques do rato. Em desenvolvimento ficou a movimentação das peças segundo esses cliques.

***
